# This notebook will teach you how to login to an existing node.
#### We will show you how to get an existing slice, get a node from the slice, get the IP and make the ssh command that you can use to access the node

#### Before we start the tutorial, we will first create a slice. Note that this is not the focus of this tutorial. There are other tutorials that show you the details of creating a slice.

In [22]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")
print(os.environ['FABRIC_CREDMGR_HOST'])

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net
beta-2.fabric-testbed.net


In [23]:
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTCNBRGEZWMOLCGYYGMNBQME4WMYJTGIZGEOBQGJQWCNJXGE2DGZB7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRTGQ2TMMBVHAYTANBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ
CILOGON_REFRESH_TOKEN environment variable: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTEZJWGZRWENBRMU4GENJTGQ4DIMRYMI2WCODFMIYDQOLBGI2TCZR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRTGQ2TKOBYHE2TIOBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ


In [24]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, 
                             refresh_token=fabric_refresh_token, project_name='all', scope='all')

### Get new Fabric Identity Token and update Fabric Refresh Token

In [25]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token
print()
print()


New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTGMJWMYZTGMZQMU3DINJXHFSDMZLDHAYTIYRVGZTDSNRVMVSWMZB7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRTGUZTAOJYHEZDQMRGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

Stored new Refresh Token
Stored 'fabric_refresh_token' (str)




Note that we need to specify a key file that we can later use to access the VM that we are reserving. We are going to store the path to that key file in the variable below.

_Also note that we assume that the private key is in a file called id_rsa, and that the public key is in a file (in the same directory) called id_rsa.pub_

In [26]:
key_path = "/home/fabric/.ssh/id_rsa"

## Create Slice

In [27]:
status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
print(f"Toplogy: {advertised_topology}")

Status: Status.OK
Toplogy: UKY: { cpu: 6/6 , core: 96/96 , ram: 1,536/1,536 G, disk: 109,600/109,600 G, unit: 3/3 }
	Components:
		GPU-Tesla T4:  GPU Tesla T4 { unit: 4/4 }
		SmartNIC-ConnectX-6:  SmartNIC ConnectX-6 { unit: 2/2 }
		SmartNIC-ConnectX-5:  SmartNIC ConnectX-5 { unit: 2/2 }
		NVME-P4510:  NVME P4510 { disk: 10,000/10,000 G, unit: 10/10 }
		SharedNIC-ConnectX-6:  SharedNIC ConnectX-6 { unit: 12/12 }
		GPU-RTX6000:  GPU RTX6000 { unit: 2/2 }
	Site Interfaces:
Links:


In [28]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType

t = ExperimentTopology()

cap = Capacities()
cap.set_fields(core=4, ram=4, disk=500)
# Add node
n1 = t.add_node(name='n2', site='UKY')
# Set properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_ubuntu_20')
# Add PCI devices
n1.add_component(ctype=ComponentType.GPU, model='Tesla T4', name='component1')

# Generate Slice Graph
slice_graph = t.serialize()

ssh_key = None
with open (key_path + ".pub", "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name='JupyterSlice1', slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(status))
print("Reservations created {}".format(reservations))

Response Status Status.OK
Reservations created [{
    "graph_node_id": "79bc3274-12fe-4ff7-bf63-0d2d338f4c5f",
    "join_state": "None_",
    "lease_end": "2021-06-13 20:50:04",
    "reservation_id": "4fda2b58-60f3-4c52-a50d-e37afdead731",
    "reservation_state": "Unknown",
    "resource_type": "VM",
    "slice_id": "8e372ebd-94f0-46c6-abb6-0a0be155ddac"
}]


# Tutorial starts here.

#### The first thing that we need to do is to get our existing slices.

In [53]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
print("Slices {}".format(slices))


Response Status Status.OK
Slices []


#### We can see that we only have one slice.

#### The next thing to do is that we want to query the contents of this slice.

#### So we grab the slice ID of the slice, and query its slivers.

In [9]:
slice_id = slices[0].slice_id

#### We have the slice ID. Now let's query the slivers.

In [10]:
status, slivers = slice_manager.slivers(slice_id=slice_id)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slivers {}".format(slivers))

Response Status Status.OK
Slivers [{
    "allocated_capacities": "{\"core\": 16, \"disk\": 500, \"ram\": 64}",
    "allocated_labels": "{\"instance\": \"instance-00000100\", \"instance_parent\": \"uky-w2\"}",
    "capacities": "{\"core\": 4, \"disk\": 500, \"ram\": 64}",
    "capacity_hints": "{\"instance_type\": \"fabric.c16.m64.d500\"}",
    "graph_node_id": "438ea273-b771-4629-a6b5-bcf5efa63b71",
    "join_state": "NoJoin",
    "lease_end": "2021-06-11 22:59:18",
    "management_ip": "128.163.179.56",
    "name": "n2",
    "reservation_id": "baa7b43a-cb34-402f-95b5-10697c9b7358",
    "reservation_state": "Active",
    "resource_type": "VM",
    "site": "UKY",
    "slice_id": "f322554b-3ff4-4510-989b-d05d9ddc06a1"
}]


The IP that we can use to access our virtual machine is in the `management_ip` variable. So this is how we can obtain it.

In [11]:
IPaddress = slivers[0].management_ip

We have the IP. Now we print the SSH command that we can use to access the machine. We need to use the key path variable that we've defined at the time of slice creation. We have the path to our key stored in the "key_path" variable.

In [12]:
print("ssh -i " + key_path + " ubuntu@" + IPaddress)

ssh -i /home/fabric/.ssh/id_rsa ubuntu@128.163.179.56


You can use the command above in a terminal to access the VM.